In [1]:
from datasets import load_dataset
import json

/home/whlin/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_files = {"train": "./data/train.json", "test": "./data/valid.json"}
qa_dataset = load_dataset("json", data_files=data_files)



Using custom data configuration default-bf2c0e59b3d5d02d
Found cached dataset json (/home/whlin/.cache/huggingface/datasets/json/default-bf2c0e59b3d5d02d/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 2/2 [00:00<00:00, 1066.71it/s]


In [3]:
with open("./data/context.json") as f:
    context_data = json.load(f)

question_column_name = "question" if "question" in column_names else column_names[0]
context_column_name = "context" if "context" in column_names else column_names[1]
answer_column_name = "answers" if "answers" in column_names else column_names[2]

In [4]:
def preprocess(example):
    data = dict()
    data['context'] = context_data[example['relevant']]
    data['answers'] = {"answer_start": [example['answer']['start']],
                       "text": [example['answer']['text']]}
    
    return data

In [5]:
test_dataset = qa_dataset['train'].select(range(10))
tokenized_dataset = test_dataset.map(preprocess, remove_columns=['answer', 'paragraphs', 'relevant'])

Loading cached processed dataset at /home/whlin/.cache/huggingface/datasets/json/default-bf2c0e59b3d5d02d/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-d1b28f5340a65c13.arrow


In [6]:
print(tokenized_dataset)
print(tokenized_dataset[0])

Dataset({
    features: ['id', 'question', 'context', 'answers'],
    num_rows: 10
})
{'id': '593f14f960d971e294af884f0194b3a7', 'question': '舍本和誰的數據能推算出連星的恆星的質量？', 'context': '在19世紀雙星觀測所獲得的成就使重要性也增加了。在1834年，白塞爾觀測到天狼星自行的變化，因而推測有一顆隱藏的伴星；愛德華·皮克林在1899年觀測開陽週期性分裂的光譜線時發現第一顆光譜雙星，週期是104天。天文學家斯特魯維和舍本·衛斯里·伯納姆仔細的觀察和收集了許多聯星的資料，使得可以從被確定的軌道要素推算出恆星的質量。第一個獲得解答的是1827年由菲利克斯·薩瓦里透過望遠鏡的觀測得到的聯星軌道。對恆星的科學研究在20世紀獲得快速的進展，相片成為天文學上很有價值的工具。卡爾·史瓦西發現經由比較視星等和攝影星等的差別，可以得到恆星的顏色和它的溫度。1921年，光電光度計的發展可以在不同的波長間隔上非常精密的測量星等。阿爾伯特·邁克生在虎克望遠鏡第一次使用干涉儀測量出恆星的直徑。', 'answers': {'answer_start': [108], 'text': ['斯特魯維']}}


In [7]:
tokenized_dataset[0]['context'].index("斯特魯維")

108

In [11]:
qa_dataset.filter(lambda x: x["answer"] is None)

Loading cached processed dataset at /home/whlin/.cache/huggingface/datasets/json/default-bf2c0e59b3d5d02d/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-8fd461a3c78b9f14.arrow
Loading cached processed dataset at /home/whlin/.cache/huggingface/datasets/json/default-bf2c0e59b3d5d02d/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-a7fdb5861a5155b0.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'paragraphs', 'relevant', 'answer'],
        num_rows: 0
    })
    test: Dataset({
        features: ['id', 'question', 'paragraphs', 'relevant', 'answer'],
        num_rows: 0
    })
})

all question has answer

In [15]:
max_length = 0
sentence = ""
for answer in qa_dataset['test']['answer']:
    if len(answer['text']) > max_length:
        max_length = len(answer['text'])
        sentence = answer['text']

print(max_length)
print(sentence)

63
氧化磷酸化中，通過如檸檬酸循環等代謝途徑，電子從被消化吸收的食物分子上轉移到氧氣上，並將產生的能量以三磷酸腺苷的方式儲存起來。


In [14]:
max_start = 0
sentence = ""
for answer in qa_dataset['train']['answer']:
    if answer['start'] > max_start:
        max_start = answer['start']
        sentence = answer['text']

print(max_start)
print(sentence)

1254
1120公尺


# try xlnet

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

In [4]:
tokenizer.padding_side

'left'

In [10]:
from datasets import load_dataset

raw_datasets = load_dataset("squad")

Found cached dataset squad (/home/whlin/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00,  7.13it/s]


In [11]:
sequence = ["hello world peko peko hello world peko peko", "my name is pekora cha hello world peko peko hello world peko peko"]
answer = ["peko hello world peko peko hello world peko peko", "peko hello world peko peko hello world peko peko"]
output = tokenizer(
    raw_datasets["train"][2:6]["context"],
    raw_datasets["train"][2:6]["question"],
    truncation='only_first',
    max_length=384,
    stride=128,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

In [19]:
output[0]

Encoding(num_tokens=182, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [21]:
print(raw_datasets["train"][2]["context"])
print(raw_datasets["train"][2]["question"])
for i in range(1):
    for key in output.keys():
        print(key, ": ", output[key][i])
    print(output.tokens(i))

Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
The Basilica of the Sacred heart at Notre Dame is beside to which structure?
input_ids :  [31571, 111, 19, 18, 297, 51, 24, 2335, 1542, 9, 79, 4457, 18, 5849, 5087, 26, 23, 1306, 17, 10781, 27, 24, 5540, 8928, 20, 18, 9225, 2198, 9, 20328, 25, 605, 20, 18, 5849, 5087, 21, 3077, 36, 19, 27, 24, 8177, 892